## Libraries and Functions

In [27]:
## ML
from sklearn.preprocessing import StandardScaler
from skforecast.recursive import ForecasterRecursive

#Ingestion
import psycopg2
import numpy as np
## Data Manipulation
from datetime import timedelta
import pandas as pd
from skforecast.recursive import ForecasterRecursive
import xgboost as xg

#Os
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env',override=True)

True

In [28]:
def format_date(dt:pd.Series) -> pd.Series:
    return dt.date()

def fill_scrapping(df:pd.DataFrame,columns:list) -> pd.DataFrame:
    for column in columns:
        df[column] = df[column].ffill() 
        df[column] = df[column].bfill() 
    df["category"] = (df["category"].str.replace("\n","",regex=False))
    return df

def formating_df(df: pd.DataFrame) -> pd.DataFrame:
    indexs = df.index
    siz = len(df.index.names)
    new_df = dict()
    for name,i in zip(df.index.names,range(0,siz)):
        column = indexs.get_level_values(i)
        new_df[name] = column
    vol = len(df.columns)
    for names,i in zip(df.columns,range(vol)):
        temp_values = df.iloc[:,i].values
        new_df[names] = temp_values
    return pd.DataFrame(new_df)

def agg_type(aggregation):
    if aggregation == "max":
        return "temp_max"
    elif aggregation == "min":
        return "temp_min"
    else:
        return "actual_temp"
def agg_values(df,aggregation):
    agg = agg_type(aggregation)
    return df.loc[:,["date_scrap","city_name","category",f"{agg}"]]

def standard_df(df,aggregation,std_scaler):
    df = agg_values(df.copy(),aggregation)
    y = df.iloc[:,-1].values.reshape(-1,1)        
    df.iloc[:,-1] = std_scaler.fit_transform(y)
    df = df.rename(columns={"date_scrap":"dt"})
    df = df.set_index("dt")
    return df

def city_df(df,city):
    df = df.loc[df["city_name"]==city]
    df = df.asfreq('D').drop("city_name",axis=1)
    return df
    
def get_categorical(df:pd.DataFrame) -> pd.DataFrame:
    x = pd.get_dummies(df,columns=["category"],dtype=int)
    return x


In [29]:
def train_test(df:pd.DataFrame,aggregation):
    source_start = pd.to_datetime("2025-06-03")
    final_train_source = source_start + timedelta(days=int(len(df)*0.8))
    agg = agg_type(aggregation)
    y_train = df.loc[:final_train_source,agg]
    y_test = df.loc[final_train_source + timedelta(days=1):,agg]
    x_train = df.loc[:final_train_source ,df.columns != agg]
    x_test =  df.loc[final_train_source + timedelta(days=1):,df.columns != agg]
    return y_train,y_test,x_train,x_test

## Script

In [30]:
ip_sql = os.getenv("ip_sql_gcp")
user =  os.getenv("user_gcp")
password = os.getenv("password_gcp")
database = os.getenv("database")


In [31]:
con = psycopg2.connect("dbname={dbname} user={user} password={password} host={host} port=5432".format(dbname=database,user=user,password=password,host=ip_sql))

## Feature Engineering

In [32]:
scrapping_gen = pd.read_sql("SELECT * FROM public.scrapping",con=con).drop("url",axis=1)
weather_df = pd.read_sql("SELECT * FROM public.weather",con=con)
weather_df["date_scrap"] = weather_df["actual_date"].apply(lambda x: format_date(x))
weather_df = weather_df.groupby(["city_name","date_scrap"]).agg({"actual_temp":"mean","temp_min":"min","temp_max":"max"})
weather_df = formating_df(weather_df)

weather_df = weather_df.merge(
    scrapping_gen,
    how="left",
    on="date_scrap"
)

daily_weather = fill_scrapping(weather_df,["category","text"])

/tmp/ipykernel_126807/3830060771.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scrapping_gen = pd.read_sql("SELECT * FROM public.scrapping",con=con).drop("url",axis=1)
/tmp/ipykernel_126807/3830060771.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weather_df = pd.read_sql("SELECT * FROM public.weather",con=con)


### Testing

In [34]:
scaler_actualtemp = StandardScaler()
daily_weather_actualtemp = standard_df(daily_weather,"mean",scaler_actualtemp)
daily_weather_actualtemp = get_categorical(city_df(daily_weather_actualtemp,"São José dos Campos"))

scaler_mintemp = StandardScaler()

daily_weather_min = standard_df(daily_weather,"min",scaler_mintemp)
daily_weather_min = get_categorical(city_df(daily_weather_min,"São José dos Campos"))

scaler_maxtemp = StandardScaler()

daily_weather_max = standard_df(daily_weather,"max",scaler_maxtemp)
daily_weather_max = get_categorical(city_df(daily_weather_max,"São José dos Campos"))




## ML

### Hyperparameter Tuning(TempMax)

In [35]:
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import TimeSeriesFold 


In [36]:
forecaster = ForecasterRecursive(
    regressor=xg.XGBRegressor(random_state=123),
    lags=3 
)

cv = TimeSeriesFold(
         steps              = 2,
         initial_train_size = 24,
         refit              = False
     )

lags_grid = [1,2,3,4,5,6,7,10]


param_grid = {
    'n_estimators': [2,5,10,20,50,100,125],
    'max_depth': [1,2,5,10, 15],
    'reg_lambda': [0.1, 1.0],
    'reg_alpha':[0.1,1.0]
}

In [37]:
results = grid_search_forecaster(
              forecaster    = forecaster,
              y             = daily_weather_max["temp_max"],
              param_grid    = param_grid,
              lags_grid     = lags_grid,
              exog=daily_weather_max.loc[:,(daily_weather_max.columns != "temp_max")],
              cv            = cv,
              metric        = 'mean_squared_error',
              return_best   = False,
              n_jobs        = 'auto',
              verbose       = False,
              show_progress = True
          )

lags grid:   0%|          | 0/8 [00:00<?, ?it/s]

params grid:   0%|          | 0/140 [00:00<?, ?it/s]

In [38]:
results

,lags,lags_label,params,mean_squared_error,max_depth,n_estimators,reg_alpha,reg_lambda
0,"[1, 2]","[1, 2]","{'max_depth': 5, 'n_estimators': 10, 'reg_alph...",0.434963,5.0,10.0,0.1,0.1
1,"[1, 2, 3, 4]","[1, 2, 3, 4]","{'max_depth': 1, 'n_estimators': 125, 'reg_alp...",0.437650,1.0,125.0,0.1,0.1
2,"[1, 2]","[1, 2]","{'max_depth': 10, 'n_estimators': 10, 'reg_alp...",0.440234,10.0,10.0,0.1,0.1
3,"[1, 2]","[1, 2]","{'max_depth': 15, 'n_estimators': 10, 'reg_alp...",0.440234,15.0,10.0,0.1,0.1
4,"[1, 2]","[1, 2]","{'max_depth': 15, 'n_estimators': 20, 'reg_alp...",0.441840,15.0,20.0,0.1,0.1
...,...,...,...,...,...,...,...,...
1115,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 10, 'n_estimators': 100, 'reg_al...",2.805646,10.0,100.0,0.1,0.1
1116,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 10, 'n_estimators': 50, 'reg_alp...",2.805646,10.0,50.0,0.1,0.1
1117,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 5, 'n_estimators': 125, 'reg_alp...",2.805646,5.0,125.0,0.1,0.1
1118,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 5, 'n_estimators': 100, 'reg_alp...",2.805646,5.0,100.0,0.1,0.1


### Final Model (TempMax)

In [39]:
y_train,y_test,x_train,x_test = train_test(daily_weather_max,"max")

forecaster = ForecasterRecursive(
    regressor= xg.XGBRegressor(max_depth=1,n_estimators=125,reg_alpha=0.1,reg_lambda=0.1),
    lags=4,
)

forecaster.fit(
    y = y_train,
    exog = x_train
)

predictions = forecaster.predict(
    steps=5,
    exog = x_test
)

In [40]:
scaler_maxtemp.inverse_transform(y_test.values.reshape(-1,1))

array([[25.27],
       [26.27],
       [29.27],
       [25.27],
       [19.27]])

In [41]:
scaler_maxtemp.inverse_transform(predictions.values.reshape(-1,1))

array([[23.57350313],
       [26.09026361],
       [25.86459158],
       [25.6746239 ],
       [25.72423247]])

### HyperParameter Tunning (TempMin)

In [42]:
forecaster = ForecasterRecursive(
    regressor=xg.XGBRegressor(random_state=123),
    lags=3 
)

cv = TimeSeriesFold(
         steps              = 2,
         initial_train_size = 24,
         refit              = False
     )

lags_grid = [1,2,3,4,5,6,7,10]


param_grid = {
    'n_estimators': [2,5,10,20,50,100,125],
    'max_depth': [1,2,5,10, 15],
    'reg_lambda': [0.1, 1.0],
    'reg_alpha':[0.1,1.0]
}

In [43]:
results = grid_search_forecaster(
              forecaster    = forecaster,
              y             = daily_weather_min["temp_min"],
              param_grid    = param_grid,
              lags_grid     = lags_grid,
              exog=daily_weather_min.loc[:,(daily_weather_max.columns != "temp_min")],
              cv            = cv,
              metric        = 'mean_squared_error',
              return_best   = False,
              n_jobs        = 'auto',
              verbose       = False,
              show_progress = True
          )

lags grid:   0%|          | 0/8 [00:00<?, ?it/s]

params grid:   0%|          | 0/140 [00:00<?, ?it/s]

In [44]:
results

,lags,lags_label,params,mean_squared_error,max_depth,n_estimators,reg_alpha,reg_lambda
0,[1],[1],"{'max_depth': 2, 'n_estimators': 100, 'reg_alp...",0.013809,2.0,100.0,0.1,0.1
1,[1],[1],"{'max_depth': 2, 'n_estimators': 125, 'reg_alp...",0.013809,2.0,125.0,0.1,0.1
2,[1],[1],"{'max_depth': 2, 'n_estimators': 50, 'reg_alph...",0.013809,2.0,50.0,0.1,0.1
3,[1],[1],"{'max_depth': 2, 'n_estimators': 125, 'reg_alp...",0.013920,2.0,125.0,0.1,1.0
4,[1],[1],"{'max_depth': 2, 'n_estimators': 100, 'reg_alp...",0.013920,2.0,100.0,0.1,1.0
...,...,...,...,...,...,...,...,...
1115,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 1, 'n_estimators': 2, 'reg_alpha...",0.935559,1.0,2.0,1.0,1.0
1116,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 2, 'n_estimators': 2, 'reg_alpha...",0.986140,2.0,2.0,0.1,1.0
1117,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 10, 'n_estimators': 2, 'reg_alph...",1.052830,10.0,2.0,0.1,1.0
1118,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 5, 'n_estimators': 2, 'reg_alpha...",1.052830,5.0,2.0,0.1,1.0


### FinalModel TempMin

In [45]:
y_train,y_test,x_train,x_test = train_test(daily_weather_min,"min")

forecaster = ForecasterRecursive(
    regressor= xg.XGBRegressor(max_depth=2,n_estimators=100,reg_alpha=0.1,reg_lambda=0.1),
    lags=4,
)

forecaster.fit(
    y = y_train,
    exog = x_train
)

predictions = forecaster.predict(
    steps=5,
    exog = x_test
)

In [46]:
scaler_mintemp.inverse_transform(y_test.values.reshape(-1,1))

array([[15.27],
       [17.27],
       [14.27],
       [14.27],
       [16.27]])

In [47]:
scaler_mintemp.inverse_transform(predictions.values.reshape(-1,1))

array([[13.64246481],
       [11.08122338],
       [ 9.43691786],
       [ 9.99475953],
       [10.87907393]])

### HyperParameter Tunning (ActualTemp)

In [48]:
forecaster = ForecasterRecursive(
    regressor=xg.XGBRegressor(random_state=123),
    lags=3 
)

cv = TimeSeriesFold(
         steps              = 2,
         initial_train_size = 24,
         refit              = False
     )

lags_grid = [1,2,3,4,5,6,7,10]


param_grid = {
    'n_estimators': [2,5,10,20,50,100,125],
    'max_depth': [1,2,5,10, 15],
    'reg_lambda': [0.1, 1.0],
    'reg_alpha':[0.1,1.0]
}

In [49]:
results = grid_search_forecaster(
              forecaster    = forecaster,
              y             = daily_weather_actualtemp["actual_temp"],
              param_grid    = param_grid,
              lags_grid     = lags_grid,
              exog=daily_weather_min.loc[:,(daily_weather_actualtemp.columns != "actual_temp")],
              cv            = cv,
              metric        = 'mean_squared_error',
              return_best   = False,
              n_jobs        = 'auto',
              verbose       = False,
              show_progress = True
          )

lags grid:   0%|          | 0/8 [00:00<?, ?it/s]

params grid:   0%|          | 0/140 [00:00<?, ?it/s]

In [50]:
results

,lags,lags_label,params,mean_squared_error,max_depth,n_estimators,reg_alpha,reg_lambda
0,"[1, 2]","[1, 2]","{'max_depth': 1, 'n_estimators': 50, 'reg_alph...",0.386195,1.0,50.0,0.1,1.0
1,"[1, 2]","[1, 2]","{'max_depth': 15, 'n_estimators': 125, 'reg_al...",0.397114,15.0,125.0,0.1,0.1
2,"[1, 2]","[1, 2]","{'max_depth': 15, 'n_estimators': 100, 'reg_al...",0.397114,15.0,100.0,0.1,0.1
3,"[1, 2]","[1, 2]","{'max_depth': 10, 'n_estimators': 100, 'reg_al...",0.397114,10.0,100.0,0.1,0.1
4,"[1, 2]","[1, 2]","{'max_depth': 10, 'n_estimators': 125, 'reg_al...",0.397114,10.0,125.0,0.1,0.1
...,...,...,...,...,...,...,...,...
1115,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 15, 'n_estimators': 10, 'reg_alp...",3.432761,15.0,10.0,0.1,0.1
1116,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 5, 'n_estimators': 10, 'reg_alph...",3.432761,5.0,10.0,0.1,0.1
1117,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 15, 'n_estimators': 20, 'reg_alp...",3.442104,15.0,20.0,0.1,0.1
1118,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","{'max_depth': 5, 'n_estimators': 20, 'reg_alph...",3.442104,5.0,20.0,0.1,0.1


### FinalModel (ActualTemp)

In [51]:
y_train,y_test,x_train,x_test = train_test(daily_weather_min,"min")

forecaster = ForecasterRecursive(
    regressor= xg.XGBRegressor(max_depth=1,n_estimators=50,reg_alpha=0.1,reg_lambda=1.0),
    lags=2,
)

forecaster.fit(
    y = y_train,
    exog = x_train
)

predictions = forecaster.predict(
    steps=5,
    exog = x_test
)

In [52]:
scaler_actualtemp.inverse_transform(y_test.values.reshape(-1,1))

array([[19.72062793],
       [21.11897831],
       [19.02145274],
       [19.02145274],
       [20.41980312]])

In [53]:
scaler_actualtemp.inverse_transform(predictions.values.reshape(-1,1))

array([[17.77374963],
       [16.1917254 ],
       [15.09223522],
       [16.67425736],
       [16.67425736]])